# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [4]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [5]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [6]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())

In [7]:
print(len(train_doc))
print(len(val_doc))

663
100


In [8]:
def standardize(img):
    mean = np.mean(img)
    std = np.std(img)
    img = (img - mean) / std
    return img

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [9]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,3,4,5,6,7,8,10,12,14,15,16,17,18,19,20,22,24,25,26,27,28,29] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,25,80,80,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(80,80)).astype(np.float32)
                    else:
                        image = imresize(image[:,20:340,:],(80,80)).astype(np.float32)
                        
                    batch_data[folder,idx,:,:,0] = standardize(image[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = standardize(image[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = standardize(image[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,25,80,80,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(80,80)).astype(np.float32)
                    else:
                        image = imresize(image[:,20:340,:],(80,80)).astype(np.float32)

                    batch_data[folder,idx,:,:,0] = standardize(image[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = standardize(image[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = standardize(image[:,:,2]) #normalise and feed in the image

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [10]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 40 # choose the number of epochs
print ('# epochs =', num_epochs)
batch_size = 32 #experiment with the batch size

# training sequences = 663
# validation sequences = 100
# epochs = 40


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [21]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
model = Sequential()
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(25,80,80,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [22]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 25, 80, 80, 64)    5248      
_________________________________________________________________
batch_normalization_5 (Batch (None, 25, 80, 80, 64)    256       
_________________________________________________________________
activation_5 (Activation)    (None, 25, 80, 80, 64)    0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 12, 40, 80, 64)    0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 12, 40, 80, 128)   221312    
_________________________________________________________________
batch_normalization_6 (Batch (None, 12, 40, 80, 128)   512       
_________________________________________________________________
activation_6 (Activation)    (None, 12, 40, 80, 128)   0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [23]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [24]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [25]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [26]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =   Project_data/val ; batch size =Epoch 1/40
Project_data/train ; batch size = 32
 32


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/21 [=>............................] - ETA: 3:06 - loss: 4.8515 - categorical_accuracy: 0.1875

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 145s 7s/step - loss: 3.0982 - categorical_accuracy: 0.2941 - val_loss: 1.3576 - val_categorical_accuracy: 0.4400

Epoch 00001: saving model to model_init_2020-02-0414_43_47.418584/model-00001-3.10798-0.29563-1.35764-0.44000.h5
Epoch 2/40
21/21 [==============================] - 64s 3s/step - loss: 1.7205 - categorical_accuracy: 0.3815 - val_loss: 1.2209 - val_categorical_accuracy: 0.4600

Epoch 00002: saving model to model_init_2020-02-0414_43_47.418584/model-00002-1.72223-0.38311-1.22094-0.46000.h5
Epoch 3/40
21/21 [==============================] - 67s 3s/step - loss: 1.3122 - categorical_accuracy: 0.4712 - val_loss: 1.0390 - val_categorical_accuracy: 0.6200

Epoch 00003: saving model to model_init_2020-02-0414_43_47.418584/model-00003-1.31498-0.47059-1.03896-0.62000.h5
Epoch 4/40
21/21 [==============================] - 69s 3s/step - loss: 1.2065 - categorical_accuracy: 0.5282 - val_loss: 0.9657 - val_categorical_accuracy: 0.6200

Epoch 00004

21/21 [==============================] - 68s 3s/step - loss: 0.4092 - categorical_accuracy: 0.8366 - val_loss: 0.6620 - val_categorical_accuracy: 0.7100

Epoch 00029: saving model to model_init_2020-02-0414_43_47.418584/model-00029-0.41139-0.83560-0.66198-0.71000.h5
Epoch 30/40
21/21 [==============================] - 69s 3s/step - loss: 0.4741 - categorical_accuracy: 0.8081 - val_loss: 0.6593 - val_categorical_accuracy: 0.7200

Epoch 00030: saving model to model_init_2020-02-0414_43_47.418584/model-00030-0.47422-0.80845-0.65926-0.72000.h5
Epoch 31/40
21/21 [==============================] - 68s 3s/step - loss: 0.4605 - categorical_accuracy: 0.8197 - val_loss: 0.6581 - val_categorical_accuracy: 0.7100

Epoch 00031: saving model to model_init_2020-02-0414_43_47.418584/model-00031-0.46210-0.81900-0.65810-0.71000.h5
Epoch 32/40
21/21 [==============================] - 68s 3s/step - loss: 0.4658 - categorical_accuracy: 0.8310 - val_loss: 0.6567 - val_categorical_accuracy: 0.7100

Epoch 000

### Observations:

- As we can see from above model that it is overfitting. We have got categorical_accuracy: 0.8279 and val_categorical_accuracy: 0.7200. There is 10% difference between train and validation accuracies.
- Currently we are using 25 frames with size 80*80 and 3 channels as an input.
- 4 convolutional layer followed by batch normalization, maxpooling.
- Later it is flattened follwoed by dropout and fed into Dense layer with 5 classification output.
- This model is giving good accuracy but seems overfit. Though it has trainanble parameters as 9.5 million which is good for webcam hosting as well.

### Lets add dropout after third CONVO 3D layer to reduce overfit and see if our accuracy is still good.

### Model 2: CONVO 3D

In [11]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
model = Sequential()
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(25,80,80,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))
#model.add(Dropout(0.25))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
#model.add(Dropout(0.25))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model.add(Dropout(0.25))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [12]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 25, 80, 80, 64)    5248      
_________________________________________________________________
batch_normalization_5 (Batch (None, 25, 80, 80, 64)    256       
_________________________________________________________________
activation_5 (Activation)    (None, 25, 80, 80, 64)    0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 12, 40, 80, 64)    0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 12, 40, 80, 128)   221312    
_________________________________________________________________
batch_normalization_6 (Batch (None, 12, 40, 80, 128)   512       
_________________________________________________________________
activation_6 (Activation)    (None, 12, 40, 80, 128)   0         
__________

In [13]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [14]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 32
Source path =  Project_data/train ; batch size = 32
Epoch 1/40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/

21/21 [==============================] - 87s 4s/step - loss: 3.4121 - categorical_accuracy: 0.2401 - val_loss: 1.3190 - val_categorical_accuracy: 0.4800

Epoch 00001: saving model to model_init_2020-02-0416_03_27.473235/model-00001-3.41417-0.23982-1.31900-0.48000.h5
Epoch 2/40
21/21 [==============================] - 63s 3s/step - loss: 1.8373 - categorical_accuracy: 0.3570 - val_loss: 1.2891 - val_categorical_accuracy: 0.5100

Epoch 00002: saving model to model_init_2020-02-0416_03_27.473235/model-00002-1.83536-0.35596-1.28906-0.51000.h5
Epoch 3/40
21/21 [==============================] - 67s 3s/step - loss: 1.5333 - categorical_accuracy: 0.3774 - val_loss: 1.2372 - val_categorical_accuracy: 0.4600

Epoch 00003: saving model to model_init_2020-02-0416_03_27.473235/model-00003-1.53548-0.37557-1.23721-0.46000.h5
Epoch 4/40
21/21 [==============================] - 68s 3s/step - loss: 1.3458 - categorical_accuracy: 0.4716 - val_loss: 1.1761 - val_categorical_accuracy: 0.6000

Epoch 00004:

21/21 [==============================] - 68s 3s/step - loss: 0.6040 - categorical_accuracy: 0.7709 - val_loss: 0.6507 - val_categorical_accuracy: 0.7700

Epoch 00030: saving model to model_init_2020-02-0416_03_27.473235/model-00030-0.60515-0.77074-0.65069-0.77000.h5
Epoch 31/40
21/21 [==============================] - 68s 3s/step - loss: 0.6186 - categorical_accuracy: 0.7445 - val_loss: 0.6515 - val_categorical_accuracy: 0.7800

Epoch 00031: saving model to model_init_2020-02-0416_03_27.473235/model-00031-0.61966-0.74510-0.65152-0.78000.h5
Epoch 32/40
21/21 [==============================] - 67s 3s/step - loss: 0.5933 - categorical_accuracy: 0.7858 - val_loss: 0.6458 - val_categorical_accuracy: 0.7600

Epoch 00032: saving model to model_init_2020-02-0416_03_27.473235/model-00032-0.59318-0.78582-0.64580-0.76000.h5
Epoch 33/40
21/21 [==============================] - 67s 3s/step - loss: 0.5979 - categorical_accuracy: 0.7573 - val_loss: 0.6303 - val_categorical_accuracy: 0.7900

Epoch 000

### Observations of Model 2 (CONVO-3D):

- So finally we have got very good model using CONVO 3D with below parameters:
1. We have used 25 frames of size 80*80 with 3 channels. 
2. Frame size was reduced/cropped to 80*80 to extract fine features only.
3. As we can see we got ***categorical_accuracy: 0.7897*** and ***val_categorical_accuracy: 0.7700***
4. This accuracy we got after trying multiple models with different parameters.
5. Total trainable parameters ***Trainable params: 9,439,365*** which is even good for webcam hosting.
6. This model has 4 convolutional 3D layer followed by batch normalization, maxpooling.
7. 0.25 dropout has been added after third convolution layer to reduce overfitting.
8. Later it is flattened follwoed by dropout and fed into Dense layer with 5 classification outputs.